# CarpeTrip bot

#### Table of Contents  
1. [Scope definition](#choosing-destination)
2. [Venues selection](#choosing-venues)
3. [Hotels selection](#choosing-hotels)
4. [Scraping hotel prices from Trip Advisor](#scraping-prices)
5. [Vacation planning](#vacation-planning)
6. [Cost of transport calculation](#transport)
7. [Total cost of hotel calculation](#total-cost)
8. [Evaluation of results](#results)

This is a *Proof of Concept* about the backend for a travel advisor bot. The solution aims at:
- helping the user identify the venues he will be visiting during his journey,
- making a detailed plan (day by day) of the vacation,
- finding the optimal accomodation that minimizes the total cost of travelling + overnight stay (thus allowing access to higher value hotels while spending less in some cases)

## 1. Scope definition <a name="choosing-destination"></a> 

For starters, we will dress the part of the travel agency and provide our users with a list of sights to visit in the city of interest. For this proof-of-concept, we will consider the use case of a person travelling to Manhattan on vacation.

In [62]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium
from itertools import product
import requests
import numpy as np
from lxml import html
import re
from pandas.io.json import json_normalize
import pandas as pd
from tqdm import tqdm
from math import ceil, floor
import time

Let's get the coordinates and the boundaries of the area of interest:

In [2]:
address='Manhattan, New York, Ny'
geolocator = Nominatim(user_agent="city_explorer")
location = geolocator.geocode(address)
location.raw

{'boundingbox': ['40.6839411', '40.8804489', '-74.0472219', '-73.9061585'],
 'class': 'boundary',
 'display_name': 'Manhattan, New York County, New York, United States of America',
 'icon': 'https://nominatim.openstreetmap.org/images/mapicons/poi_boundary_administrative.p.20.png',
 'importance': 0.9854391745917972,
 'lat': '40.7896239',
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'lon': '-73.9598939',
 'osm_id': 8398124,
 'osm_type': 'relation',
 'place_id': 236223885,
 'type': 'administrative'}

In [3]:
lat_center, long_center = location.latitude, location.longitude
bounds = [eval(x) for x in location.raw['boundingbox']]

In order to visualize the scope of this use case we can draw a map with the folium library

In [4]:
map_area = folium.Map(location=[lat_center, long_center], zoom_start=11)

In [5]:
bbox = list(product((bounds[0],bounds[1]), (bounds[2],bounds[3])))

folium.Rectangle(
        bbox,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(map_area)
map_area

## 2. Venues selection <a name="choosing-venues"></a> 

Once we have established the destination, we demand our user to select a list of venues categories he is interested in visiting. For example, he could be asked to choose some categories among the following:
- museum
- stadium
- bridge
- park
- church
- scenic lookout
- monument

Once the selection is completed, a GET request is made to the Foursquare API in order to retrieve all venues from the chosen categories within the boundaries of Manhattan.

In [6]:
CLIENT_ID = 'HJOMTRFDACPN3RSI4US2CAOB0KYKK4HOVM2CLVXIGBG1GEDP'
CLIENT_SECRET = 'SKXEO4FJB1NM2W5YVF30XYA3HLHEF1HBN04SSBH3Y3Y0BQLM'
VERSION = '20180605'

categories_dict = {
    'Museum': '4bf58dd8d48988d181941735',
    'Stadium': '4bf58dd8d48988d184941735',
    'Bridge': '4bf58dd8d48988d1df941735',
    'Park': '4bf58dd8d48988d163941735',
    'Church': '4bf58dd8d48988d132941735',
    'Scenic Lookout': '4bf58dd8d48988d165941735',
    'Monument': '4bf58dd8d48988d12d941735'
}

Suppose our user is interested in:
- museums
- stadiums
- look-outs
- monuments

In [7]:
chosen_categories = ['Museum', 'Stadium', 'Scenic Lookout', 'Monument']
category_ids = [categories_dict[cat] for cat in ['Museum', 'Stadium', 'Scenic Lookout', 'Monument']]

We can generate the Foursquare url in the following way:

In [8]:
cat = '{}'.format(",".join(category_ids))
intent = 'browse'
sw = '{},{}'.format(bbox[0][0], bbox[0][1])
ne = '{},{}'.format(bbox[-1][0], bbox[-1][1])
limit = 100

# create the API request URL
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&sw={}&ne={}&intent={}&categoryId={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION,
        sw,
        ne,
        intent, 
        cat, 
        limit)

Before moving on, let's define a function to call said url and parse the response JSON into a pandas dataframe:

In [9]:
def foursquare_to_df(url):
    venues_list=[]
    
    # make the GET request
    results = requests.get(url).json()['response']['venues']
    
    # return only relevant information for each cityby venue
    venues_list.append([(
        v['name'], 
        v['location']['lat'], 
        v['location']['lng'],  
        v['categories'][0]['name']) for v in results])

    venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    venues.columns = ['Name', 
                  'Latitude', 
                  'Longitude', 
                  'Category']
    
    return venues

Now it's time to get the data:

In [10]:
nyvenues = foursquare_to_df(url)
print('There are {} sights corresponding to the chosen categories in Manhattan'.format(nyvenues.shape[0]))
nyvenues.head()

There are 50 sights corresponding to the chosen categories in Manhattan


,Name,Latitude,Longitude,Category
0,Brooklyn Bridge,40.705967,-73.996707,Bridge
1,East River Running Path,40.722409,-73.974076,Park
2,Manhattan Bridge,40.707661,-73.990903,Bridge
3,McCarren Park Track,40.719665,-73.951501,Track Stadium
4,Brooklyn Heights Promenade,40.698462,-73.996707,Scenic Lookout


The user is then asked to make a rough draft of the venues he would like to visit, picking from the proposed list.  
These are the must-see venues he selects:

- Brooklyn Bridge
- Empire State Building
- Madison Square Garden
- Central Park
- The Metropolitan Museum of Art
- Solomon R Guggenheim Museum
- Yankee Stadium
- Shakespeare Garden

In [11]:
sights_list = ['Brooklyn Bridge',
    'Empire State Building',
    'Madison Square Garden',
    'Central Park',
    'The Metropolitan Museum of Art (Metropolitan Museum of Art)',
    'Solomon R Guggenheim Museum',
    'Yankee Stadium',
    'Shakespeare Garden']

Furthermore, let's make the assumption that the user tells us that he would like to visit some venue not included in the list such as the Statue of Liberty.

In [12]:
add_list = ['Statue of Liberty']
rows = []

for venue in add_list:
    loc = geolocator.geocode(venue)
    lat, long = loc[1][0], loc[1][1]
    row = {'Name': venue,
           'Latitude':lat,
           'Longitude': long,
           'Category': '---'}
    
    rows.append(row)

        
venues_tmp = pd.DataFrame(rows)

# let's add the category of each place assuming we know them 
# Note: they could be asked to the user or searched thorugh Foursquare API
venues_tmp.loc[venues_tmp.Name=='Statue of Liberty', 'Category'] = 'Monument/Landmark'

# Finally let's append the temporary df to the new york dataframe
nyvenues = nyvenues.append(venues_tmp, sort=False).reset_index(drop=True)

nyvenues.tail()

,Name,Latitude,Longitude,Category
46,Greenpoint Waterfront,40.731261,-73.961202,Scenic Lookout
47,General Grant National Memorial,40.813436,-73.963067,Historic Site
48,Flatiron Building,40.741096,-73.989594,Building
49,Transmitter Pier,40.729868,-73.962376,Scenic Lookout
50,Statue of Liberty,40.689253,-74.044548,Monument/Landmark


Let's update the sights_list as well:

In [13]:
sights_list

['Brooklyn Bridge',
 'Empire State Building',
 'Madison Square Garden',
 'Central Park',
 'The Metropolitan Museum of Art (Metropolitan Museum of Art)',
 'Solomon R Guggenheim Museum',
 'Yankee Stadium',
 'Shakespeare Garden']

In [14]:
sights_list += add_list

In the following map, we can visualize the location of the chosen sights and the remaining ones.

In [15]:
venues_map = folium.Map(location=[lat_center, long_center], zoom_start=11)


for n, lat, lng, cat in zip(nyvenues.Name, nyvenues.Latitude, nyvenues.Longitude, nyvenues.Category):
    
    if n in sights_list:
        color = 'red'
        radius = 10
    else:
        color = 'blue'
        radius = 5
        
    label = folium.Popup(str(n), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=radius,
        color=color,
        fill=True,
        fill_opacity=0.7)\
        .add_child(label)\
        .add_to(venues_map)

venues_map

Let's refine the selection of sights by pointing out to the user any area that is densely populated with attractions around the places he has already chosen to visit.

In order to do so, we can apply a DBSCAN clustering algorithm to the leftovers venues from the previous request to the Foursquare API:

In [16]:
from sklearn import metrics
from sklearn import cluster
from sklearn.cluster import DBSCAN

X = nyvenues[['Latitude','Longitude']].values
X_radians = np.radians(X)
kms_per_radian = 6371.0088
kms_max_dist = 1
epsilon = kms_max_dist/kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=2, algorithm='ball_tree', metric='haversine').fit(X_radians)

In [17]:
nyvenues['Cluster'] = db.labels_

By adding the cluster information to the New York venues dataframe, it is possible to obtain:

In [18]:
nyvenues

,Name,Latitude,Longitude,Category,Cluster
0,Brooklyn Bridge,40.705967,-73.996707,Bridge,0
1,East River Running Path,40.722409,-73.974076,Park,1
2,Manhattan Bridge,40.707661,-73.990903,Bridge,0
3,McCarren Park Track,40.719665,-73.951501,Track Stadium,-1
4,Brooklyn Heights Promenade,40.698462,-73.996707,Scenic Lookout,0
5,Gantry Plaza State Park,40.746558,-73.958051,State / Provincial Park,2
6,WNYC Transmitter Park,40.729958,-73.960733,Park,3
7,Bethesda Terrace,40.773854,-73.970982,Plaza,4
8,Exchange Waterfront,40.714692,-74.033152,Waterfront,5
9,Pier I,40.779774,-73.989456,Pier,6


Thus, we consider the selected attractions and the cluster they are in and we look for further attractions in that cluster:

In [19]:
explored = []
sights_cluster = []

for sight in sights_list:

    sub = nyvenues.loc[(nyvenues.Name==sight), 'Cluster']
    if not sub.empty:
        clus= sub.values[0]
        
        if clus != -1 and clus not in explored:
            print('Found other attractions near {} you might want to check out (CLUSTER {}):\n'.format(sight, clus))
            sights_cluster.append(clus)
            
            for name in nyvenues.loc[(nyvenues.Cluster==clus) & (nyvenues.Name!=sight), 'Name']:
                print(name)
                
            explored.append(clus)
            print('===================================================\n')
            


Found other attractions near Brooklyn Bridge you might want to check out (CLUSTER 0):

Manhattan Bridge
Brooklyn Heights Promenade
Empire Fulton Ferry Park
Pier 15

Found other attractions near Empire State Building you might want to check out (CLUSTER 7):

The Vessel
Madison Square Garden
Flatiron Building

Found other attractions near The Metropolitan Museum of Art (Metropolitan Museum of Art) you might want to check out (CLUSTER 4):

Bethesda Terrace
American Museum of Natural History
Central Park - Engineers' Gate
Alice in Wonderland Statue
Belvedere Castle
The Obelisk (Cleopatra's Needle)
King Jagiello / Poland Monument



Furthermore, the clusters can be plotted on the NYC map with different colors, highlighting the venues that were already selected by use of a bigger marker. Please note that the -1 cluster corresponds to outliers.

In [21]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[lat_center, long_center], zoom_start=11)

# set color scheme for the clusters
n_clust = len(set(db.labels_))
colors_array = cm.rainbow(np.linspace(0, 1, n_clust))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map

groups = []
for i, color in enumerate(rainbow):
    groups.append(folium.FeatureGroup(name='<span style=\\"color: {};\\">cluster {}</span>'.format(color, i-1)))
    
for n, lat, lng, cat, cluster in zip(nyvenues.Name, nyvenues.Latitude, nyvenues.Longitude, nyvenues.Category, nyvenues.Cluster):
    
    if n in sights_list:
        radius = 12
    else:
        radius = 4    

    label = folium.Popup(str(n), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=radius,
        color=rainbow[cluster+1],
        fill=True,
        fill_opacity=0.7)\
        .add_child(label)\
        .add_to(groups[cluster + 1])
        
    groups[cluster].add_to(map_clusters)
        
folium.map.LayerControl('topright', collapsed=False).add_to(map_clusters)

    
map_clusters

Let's assume the user follows the advice and decides to include in his journey other attractions from cluster 2 since he will already be visiting other places near to them. He then adds
- Belvedere Castle
- The Obelisk (Cleopatra's Needle)
- Central Park - Engineers' Gate
- Alice in Wonderland Statue  



In [22]:
sights_list += ["Belvedere Castle", "The Obelisk (Cleopatra's Needle)", "Central Park - Engineers' Gate", "Alice in Wonderland Statue"]

## 3. Hotels selection <a name="choosing-hotels"></a> 

Now that we have a complete list of all the places our user intends to visit during his staying, it's possible to find the optimal location to book the hotel. Ideally, this will have to be near the center of mass of all venues in order to minimize movement between places.

In [23]:
usr_venues = nyvenues[nyvenues.Name.isin(sights_list)].reset_index(drop=True)
usr_venues.head()

,Name,Latitude,Longitude,Category,Cluster
0,Brooklyn Bridge,40.705967,-73.996707,Bridge,0
1,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4
2,Yankee Stadium,40.829869,-73.926584,Baseball Stadium,-1
3,Empire State Building,40.748600,-73.985806,Building,7
4,Central Park - Engineers' Gate,40.784075,-73.958695,Monument / Landmark,4


Thus, the *center of gravity* is obtained:

In [24]:
cm = [usr_venues.Latitude.mean(), usr_venues.Longitude.mean()]

We obtain the optimal area in which to look for hotels by considering a ball of 1km of radius around the center of gravity:

In [29]:
optimal_map = folium.Map(location=[lat_center, long_center], zoom_start=11)


for n, lat, lng, cat in zip(usr_venues.Name, usr_venues.Latitude, usr_venues.Longitude, usr_venues.Category):
        
    label = folium.Popup(str(n), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        fill=True,
        fill_opacity=0.7)\
        .add_child(label)\
        .add_to(optimal_map)

folium.Circle(
    [cm[0], cm[1]],
    radius=1000,
    color='blue',
    fill=True,
    fill_opacity=0.7).add_to(optimal_map) 

optimal_map

We can now proceed to query the Foursquare service for hotels in the optimal area:

In [30]:
radius = 1000
category = '4bf58dd8d48988d1fa931735'
limit = 100

url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION,
                cm[0],
                cm[1],
                1000, 
                category, 
                limit)


In [31]:
df_hotels = foursquare_to_df(url)
hotels_opt = list(df_hotels.Name)
df_hotels.head()

,Name,Latitude,Longitude,Category
0,Baccarat Hotel,40.760656,-73.976784,Hotel
1,Hilton (New York Hilton Midtown),40.762248,-73.979190,Hotel
2,JW Marriott Essex House New York,40.766277,-73.978572,Hotel
3,The Sherry-Netherland,40.764332,-73.972496,Hotel
4,The Plaza Hotel,40.764519,-73.974488,Hotel


## 4. Scraping hotel prices from TripAdvisor <a name="scraping-prices"></a> 

Just by looking at Trip Advisor, it's possible to find over 30 pages of hotel listings in New York complete with prices data.  
Please note that before parsing the html source code, it will be necessary to allow the execution of Javascript code that populates the elements with the corresponding prices: for this reason, we will use _selenium_ with Python which emulates a web browser and makes it possible to retrieve specific elements by class name.  
The following functions will be used to initialize the web driver from the _selenium_ package and parse the html structure behind each page.

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options  
from selenium.webdriver.support.ui import WebDriverWait
        
    
def init_web_driver():
    chrome_options = Options()  
    chrome_options.add_argument("--headless") 
    driver = webdriver.Chrome(options=chrome_options)
    
    return driver


def webdriver_scrape(driver, url):
    driver.get(url)
    
    try:
        driver.execute_script("return document.getElementsByTagName('html')[0].innerHTML")
    except Exception as exception_message:
        print(exception_message)
        
    return driver

By inspecting the source code it appears that hotel listings can be found in the class _allowEllipsis_, from which we can obtain name and price by searching for the classes _listing-title_ and _price-wrap_ respectively.  


In [38]:
def find_listing(driver):
    element = driver.find_elements_by_class_name('allowEllipsis')
    
    if element:
        return element
    else:
        return False
    

def find_name(driver):
    element = driver.find_element_by_class_name('listing-title')
    
    if element:
        return element
    else:
        return False
    

def find_price(driver):
    element = driver.find_element_by_class_name('price-wrap')
    
    if element:
        return element
    else:
        return False
    
    
def clean_price(raw_price):  
    try:
        pr_raw = raw_price.replace('\n','')\
                          .replace('\r','')\
                          .replace('\t','')        
        pr = re.search('(\d{2,5}\s*€\s*)?(\d{2,5})', pr_raw).group(2) 
        
        return pr

    except Exception as pe:
        print(pe)


def clean_name(raw_name):
    try:
        nm_raw = raw_name.replace('\n','')\
                         .replace('\r','')\
                         .replace('\t','') 
        nm_blanks = re.sub('(?i)sponsorizzato','',nm_raw)
        nm = re.sub('(^ )|( $)','',nm_blanks)
        return nm
    
    except Exception as ne:
        print(ne)


Let's consider the first listing: from the analysis of the source code it appears that the name can be then found in a child of the considered element with tag "a"  whose path is mutable. In this case:

In [39]:
trip_url = 'https://www.tripadvisor.it/Hotels-g60763-oa0-New_York_City_New_York-Hotels.html'

# initialize web driver
driver = init_web_driver()
driver = webdriver_scrape(driver, trip_url)

# parse html
sample_hotel= driver.find_elements_by_class_name("allowEllipsis")[0]
sample_name_raw = sample_hotel.find_element_by_class_name("listing-title").text
sample_price_raw = sample_hotel.find_element_by_class_name("price-wrap").text

print('Listing: {}\nPrice: {}'.format(sample_name_raw, sample_price_raw))

Listing: Sponsorizzato
Sister City
Price: 270 €
258 €


We can extend the process and integrate the previously defined functions that avoid errors in loading the pages and clean the output information, selecting only one price when there are more options.
Please note that, in order to switch page, we just need to increment by 30 (the number of listings per page) the number after _Hotels-g60763-oa_ in the url.

In [ ]:
max_page = 30
hotel_prices = {}
hotels_opt = list(df_hotels.Name)

for ii in tqdm(range(1, max_page)):
    trip_url = 'https://www.tripadvisor.it/Hotels-g60763-oa{}-New_York_City_New_York-Hotels.html'.format(ii*30)
    driver = webdriver_scrape(driver, trip_url)

    # get list of hotels in the page
    hotel_listings = driver.find_elements_by_class_name("allowEllipsis")
    
    for hotel in hotel_listings:

        hotel_name_raw = WebDriverWait(hotel, 10).until(find_name).text
        hotel_name = clean_name(hotel_name_raw)
        
        try:
            hotel_price_raw = WebDriverWait(hotel, 10).until(find_price).text

            if hotel_name in hotels_opt:
                hotel_price = clean_price(hotel_price_raw)
                df_hotels.loc[df_hotels.Name==hotel_name, 'Price'] = hotel_price
        
        except Exception as e:
            print('Price N/A for {} at page {}: {}'.format(hotel_name, ii+1, e))

    

Let's drop rows for which it was not possible to assign a price:

In [ ]:
df_price = df_hotels[~df_hotels.Price.isna()].reset_index(drop=True)
print('Number of rows: {}'.format(df_price.shape[0]))

In [ ]:
df_price.head()

Finally, we save the dataframe:

In [259]:
df_price.to_csv('./Resources/Hotels.csv', index=False)

## 5. Vacation planning <a name="vacation-planning"></a> 

In order to avoid re-computation of all prices, it may be useful to reload the saved price dataframe:

In [48]:
df_price = pd.read_csv('./Resources/Hotels.csv')

Let's take another look at the venues that were selected:

In [49]:
usr_venues

,Name,Latitude,Longitude,Category,Cluster
0,Brooklyn Bridge,40.705967,-73.996707,Bridge,0
1,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4
2,Yankee Stadium,40.829869,-73.926584,Baseball Stadium,-1
3,Empire State Building,40.748600,-73.985806,Building,7
4,Central Park - Engineers' Gate,40.784075,-73.958695,Monument / Landmark,4
5,Madison Square Garden,40.750752,-73.993542,Basketball Stadium,7
6,Alice in Wonderland Statue,40.775040,-73.966566,Outdoor Sculpture,4
7,Belvedere Castle,40.779359,-73.969032,Castle,4
8,The Obelisk (Cleopatra's Needle),40.779656,-73.965422,Monument / Landmark,4
9,Statue of Liberty,40.689253,-74.044548,Monument/Landmark,-1


In order to proceed, we ask our user information on his arrival and departure as well as if there are any limitations on the time available to visit the city: from this information we determine the number of days (and nights) to consider in the plan.  
For this use case, we assume the user lands in New York on a Friday morning and leaves on a Sunday evening, therefore he has 3 days and 2 nights to tour the city.

In [50]:
days = 3
nights = 2

At this point, we want to spread over the different days all the sights that are there to be seen:

In [51]:
surp = usr_venues.shape[0]%days
venues_per_day = usr_venues.shape[0]/days

# let's calculate how many venues are going to be visited each day by our user
venues_plan = np.array([1]*surp + [0]*(days-surp)) + np.array([floor(venues_per_day)]*days)

for ii, n_sights in enumerate(venues_plan):
    print('Sights to be visited on day {}: {}'.format(ii+1, n_sights))

Sights to be visited on day 1: 4
Sights to be visited on day 2: 3
Sights to be visited on day 3: 3


A few assumptions are made in order to evaluate the vacation plan:

-  the airport is considered to be the starting and ending point of the trip
- each day the user leaves from the hotel to visit the farthest venue 
- the user cannot visit more than a predetermined number of venues per day (if the total number is 15 in 3 days  &rarr; 5 venues per day) 

The plan is then defined by a series of logical rules depending on two scenarios:

1. the user is visiting a point from the -1 cluster (outlier):
	*carpetrip* looks for the closest point
	- if that point is also an outlier that will be the next venue in the plan
	- if the closest point belongs to a cluster, the algorithm checks if the cluster fits in the maximum number of venues that can be visited in a day
		-  if not, it will break down the cluster
		- if so, the algrithm checks if there is enough time left in the current day to visit the cluster, if not it will direct the user there on another day
2. the user is visiting a place from one of the clusters:
	*carpetrip* looks for other points in the cluster
	
	- if there is such point, then it directs the user there
	- if the venues from that cluster are over, then it looks for the closest point and repeats the process described in case 1

In [52]:
def get_airport_info(airport):
    loc = geolocator.geocode(airport)
    lat, long = loc[1][0], loc[1][1]

    airport_info = {'Name': airport,
                    'Latitude':lat,
                    'Longitude': long,
                    'Category': 'Airport'}


    df_air = pd.DataFrame([airport_info], columns=['Name', 'Latitude', 'Longitude', 'Category'])
    
    return df_air


def get_next_place(df_visit, reference_pos, criterion='closest'):
    df_visit['Distance'] = df_visit.apply(lambda x: geodesic(reference_pos, (x.Latitude, x.Longitude)).km, axis=1)
    if criterion == 'closest':
        visit = df_visit[df_visit.Distance == df_visit.Distance.min()]
    else:
        visit = df_visit[df_visit.Distance == df_visit.Distance.max()]
    
    return visit    


def tour_planner(airport_selection, hotel_selection, usr_venues, venues_plan):
    visits = pd.DataFrame()
    df_visit = usr_venues.copy()
    airport_selection['Cluster'], airport_selection['Distance'], airport_selection['Day'] = -1, 0.0, 1
    visits = visits.append(airport_selection, sort=False)
    
    for day in range(len(venues_plan)):
        criterion = 'farthest'
        currentPos = hotel_selection.copy()
        currentPos['Day'] = day + 1
        currentPos['Distance'] = geodesic((currentPos.iloc[0][1], currentPos.iloc[0][2]),\
                                              (visits.iloc[-1][1], visits.iloc[-1][2])).km
        visits = visits.append(currentPos, sort=False)
        cond = visits[(visits.Day==day+1) & (~visits.Category.isin(['Hotel', 'Airport']))].shape[0]

        while cond < venues_plan[day]:  
               
            pos0, clust0 = [(currentPos.values[0][1], currentPos.values[0][2]), currentPos.values[0][4]]
            df_current = df_visit.loc[df_visit.Cluster==clust0, :].copy()

            if clust0==-1 or (clust0>-1 and df_current.empty):
                nextPos = get_next_place(df_visit, pos0, criterion=criterion)
                criterion='closest'
                pos1, clust1 = [(nextPos.values[0][1], nextPos.values[0][2]), nextPos.values[0][4]]

                if clust1==-1:
                    currentPos = nextPos.copy()
                    currentPos['Day'] = day + 1
                    visits = visits.append(currentPos, sort=False)
                    df_visit = df_visit[~df_visit.Name.isin(currentPos.Name)]
                else:
                    cl_dim = df_visit.loc[df_visit.Cluster==clust1, :].shape[0]

                    # check if the cluster fits in one day (among those left in the trip)
                    if any([cl_dim <= lim for lim in venues_plan[day:]]):

                        # check if the cluster will fit in the current day
                        if cl_dim <= venues_plan[day]-len(visits) + 1:
                            currentPos = nextPos.copy()
                            currentPos['Day'] = day + 1
                            visits = visits.append(currentPos, sort=False)
                            df_visit = df_visit[~df_visit.Name.isin(currentPos.Name)]

                        else:
                            # before setting aside the cluster let's check how many other
                            # clusters would be postponed and if they will all fit in the future days
                            cl_dims = df_visit.groupby('Cluster').count()
                            cl_leftover = cl_dims[cl_dims['Name']>venues_plan[day]-len(visits)].sum()[0]

                            if cl_leftover < sum(venues_plan[day+1:]):
                                df_visit = df_visit[df_visit.Cluster!=clust1]

                            else:
                                currentPos = nextPos.copy()
                                currentPos['Day'] = day + 1
                                visits = visits.append(currentPos, sort=False)
                                df_visit = df_visit[~df_visit.Name.isin(currentPos.Name)]
                    else:
                        currentPos = nextPos.copy()
                        currentPos['Day'] = day + 1
                        visits = visits.append(currentPos, sort=False)
                        df_visit = df_visit[~df_visit.Name.isin(currentPos.Name)]


            else:
                nextPos = get_next_place(df_current, pos0, criterion=criterion)
                criterion='closest'
                currentPos = nextPos.copy()
                currentPos['Day'] = day + 1
                visits = visits.append(currentPos, sort=False)
                df_visit = df_visit[~df_visit.Name.isin(currentPos.Name)]

            cond = visits[(visits.Day==day+1) & (~visits.Category.isin(['Hotel', 'Airport']))].shape[0]
    
    airport_selection['Distance'] = geodesic((airport_selection.iloc[0][1], airport_selection.iloc[0][2]),\
                                              (visits.iloc[-1][1], visits.iloc[-1][2])).km
    airport_selection['Day'] = day + 1
    visits = visits.append(airport_selection, sort=False)
    cols = ['Day'] + [col for col in visits.columns if col != 'Day']
    visits = visits[cols].reset_index(drop=True)

    return visits

For starters, we will consider the case where the user stays in the first hotel of the list, the *1 Hotel Central Park*:

In [53]:
# get airport info
airport = 'JFK Airport, New York'
airport_selection = get_airport_info(airport)

# select hotel 0
hotel_selection = df_price.loc[df_price.index==0, ['Name', 'Latitude', 'Longitude', 'Category']].copy()
hotel_selection['Cluster'], hotel_selection['Distance'] = -1, 0.0

# get plan for the vacation
visits = tour_planner(airport_selection, hotel_selection, usr_venues, venues_plan)
visits

,Day,Name,Latitude,Longitude,Category,Cluster,Distance
0,1,"JFK Airport, New York",40.642948,-73.779373,Airport,-1,0.000000
1,1,1 Hotel Central Park,40.765116,-73.976485,Hotel,-1,21.483515
2,1,Statue of Liberty,40.689253,-74.044548,Monument/Landmark,-1,10.199771
3,1,Brooklyn Bridge,40.705967,-73.996707,Bridge,0,4.449106
4,1,Empire State Building,40.748600,-73.985806,Building,7,4.822967
5,1,Madison Square Garden,40.750752,-73.993542,Basketball Stadium,7,0.695654
6,2,1 Hotel Central Park,40.765116,-73.976485,Hotel,-1,2.149211
7,2,Yankee Stadium,40.829869,-73.926584,Baseball Stadium,-1,8.333166
8,2,Central Park - Engineers' Gate,40.784075,-73.958695,Monument / Landmark,4,5.762169
9,2,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4,0.625893


## 6. Cost of transport calculation <a name="transport"></a> 

The first step to evaluate the cost of taxi fares is to translate the previous df into a transactional form that highlights movements from one location to the next and their distance:

In [54]:
def get_trips(visits):
    df_left = visits.copy()
    df_right = visits.copy()
    df_right.index = [ix-1 for ix in visits.index]
    df_right = df_right[df_right.index>=0]

    df_left = df_left[df_left.columns[:-1]]
    df_left.columns = ['Day', 'Name_start', 'Latitude_start', 'Longitude_start', 'Category_start', 'Cluster_start']
    df_right = df_right[df_right.columns[1:]]
    df_right.columns = ['Name_end', 'Latitude_end', 'Longitude_end', 'Category_end', 'Cluster_end', 'Distance']

    df_rides = pd.concat([df_left, df_right], axis=1)
    df_rides.dropna(axis=0, inplace=True)
    
    return df_rides

df_rides = get_trips(visits)
df_rides

,Day,Name_start,Latitude_start,Longitude_start,Category_start,Cluster_start,Name_end,Latitude_end,Longitude_end,Category_end,Cluster_end,Distance
0,1,"JFK Airport, New York",40.642948,-73.779373,Airport,-1,1 Hotel Central Park,40.765116,-73.976485,Hotel,-1.0,21.483515
1,1,1 Hotel Central Park,40.765116,-73.976485,Hotel,-1,Statue of Liberty,40.689253,-74.044548,Monument/Landmark,-1.0,10.199771
2,1,Statue of Liberty,40.689253,-74.044548,Monument/Landmark,-1,Brooklyn Bridge,40.705967,-73.996707,Bridge,0.0,4.449106
3,1,Brooklyn Bridge,40.705967,-73.996707,Bridge,0,Empire State Building,40.748600,-73.985806,Building,7.0,4.822967
4,1,Empire State Building,40.748600,-73.985806,Building,7,Madison Square Garden,40.750752,-73.993542,Basketball Stadium,7.0,0.695654
5,1,Madison Square Garden,40.750752,-73.993542,Basketball Stadium,7,1 Hotel Central Park,40.765116,-73.976485,Hotel,-1.0,2.149211
6,2,1 Hotel Central Park,40.765116,-73.976485,Hotel,-1,Yankee Stadium,40.829869,-73.926584,Baseball Stadium,-1.0,8.333166
7,2,Yankee Stadium,40.829869,-73.926584,Baseball Stadium,-1,Central Park - Engineers' Gate,40.784075,-73.958695,Monument / Landmark,4.0,5.762169
8,2,Central Park - Engineers' Gate,40.784075,-73.958695,Monument / Landmark,4,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4.0,0.625893
9,2,The Metropolitan Museum of Art (Metropolitan M...,40.779729,-73.963416,Art Museum,4,1 Hotel Central Park,40.765116,-73.976485,Hotel,-1.0,1.962312


Now we proceed to calculate the total cost of transport under the following assumption:  
- a taxi is needed when two places are farther apart than 1 km
In order to carry out our job, we define some functions that will compose the url to query the TaxiFareFinder website for fares between two points, allow execution of the underlying code, get the cost of the cab ride and clean it.

In particular, it's worth noticing that the url ought to be in the form:  
```https://www.taxifarefinder.com/main.php?city={}&from={}&to={}&fromCoord={},{}&toCoord={},{}```  
where the first information to be filled in is the name of the city, followed by the names of the start and end places and by their coordinates.

In [55]:
def get_nytaxifare(driver, start_name, end_name, start_loc, end_loc):  
    
    try:
        city = 'NY'
        start_lat = start_loc[0]
        start_long = start_loc[1]
        end_lat = end_loc[0]
        end_long = end_loc[1]
        taxi_url = 'https://www.taxifarefinder.com/main.php?city={}&from={}&to={}&fromCoord={},{}&toCoord={},{}'.format(
                        city,
                        start_name,
                        end_name,
                        start_lat,
                        start_long,
                        end_lat,
                        end_long)

        taxi_driver = webdriver_scrape(driver, taxi_url)
        raw_fare = WebDriverWait(taxi_driver, 10).until(find_fare).text
        fare = clean_fare(raw_fare)
        
        return fare
    
    except Exception as e:
        return 0.0

    
def find_fare(driver):
    element = driver.find_element_by_class_name('fareValue')
    
    if element:
        return element
    else:
        return False
    
    

def clean_fare(raw_fare):  
    try:
        pr_raw = raw_fare.replace('\n','')\
                          .replace('\r','')\
                          .replace('\t','')        
        pr = re.search('((\d{1,4})([\.\,]\d{0,3}))', pr_raw).group(0) 
        
        return float(pr)

    except Exception as pe:
        print(pe)

For example, let's calculate the cost of going from the selected hotel to the Statue of Liberty:

In [56]:
driver = init_web_driver()
start_name, end_name = df_rides.iloc[0][1],  df_rides.iloc[0][6] 
start_loc, end_loc = (df_rides.iloc[0][2],  df_rides.iloc[0][3]), (df_rides.iloc[0][7],  df_rides.iloc[0][8])  

cab_fare = get_nytaxifare(driver, start_name, end_name, start_loc, end_loc)
cab_fare

59.74

Extending the process to all trips of the vacation:

In [78]:
def cab_rides_calc(df_rides):
    cab = df_rides.copy()
    cab['Pos_start'] = list(zip(cab['Latitude_start'], cab['Longitude_start']))
    cab['Pos_end'] = list(zip(cab['Latitude_end'], cab['Longitude_end']))
    fare_cols = ['Distance', 'Name_start', 'Name_end', 'Pos_start', 'Pos_end']
    fares = []

    for dist, sn, en, sloc, eloc in zip(*[cab[col] for col in fare_cols]):
        if dist>1:
            sn = re.sub('\((\w*|\s|\')*\)','',sn)
            en = re.sub('\((\w*|\s|\')*\)','',en)
            fare = get_nytaxifare(driver, sn, en, sloc, eloc)

        else:
            fare = 0.0

        fares.append(fare)

    df_rides['Fare'] = fares
    
    return df_rides

df_rides = cab_rides_calc(df_rides)


Here's an overview of the fares in this scenario:

In [57]:
df_rides[[el for el in df_rides.columns if not ('Category' in el or 'Latitude' in el or 'Longitude' in el)]]

,Day,Name_start,Cluster_start,Name_end,Cluster_end,Distance
0,1,"JFK Airport, New York",-1,1 Hotel Central Park,-1.0,21.483515
1,1,1 Hotel Central Park,-1,Statue of Liberty,-1.0,10.199771
2,1,Statue of Liberty,-1,Brooklyn Bridge,0.0,4.449106
3,1,Brooklyn Bridge,0,Empire State Building,7.0,4.822967
4,1,Empire State Building,7,Madison Square Garden,7.0,0.695654
5,1,Madison Square Garden,7,1 Hotel Central Park,-1.0,2.149211
6,2,1 Hotel Central Park,-1,Yankee Stadium,-1.0,8.333166
7,2,Yankee Stadium,-1,Central Park - Engineers' Gate,4.0,5.762169
8,2,Central Park - Engineers' Gate,4,The Metropolitan Museum of Art (Metropolitan M...,4.0,0.625893
9,2,The Metropolitan Museum of Art (Metropolitan M...,4,1 Hotel Central Park,-1.0,1.962312


In this case too, we save the dataframe to avoid having to repeat the calculations (which are quite demanding) in the future:

In [237]:
df_rides.to_csv('./Resources/df_fares.csv', index=False)

## 7. Total cost of hotel calculation <a name="total-cost"></a> 

Let's reload rides for the first hotels...

In [59]:
df_rides = pd.read_csv('./Resources/df_fares.csv')

... and go back to the hotel prices dataframe:

In [60]:
df_price

,Name,Latitude,Longitude,Category,Price
0,1 Hotel Central Park,40.765116,-73.976485,Hotel,642
1,JW Marriott Essex House New York,40.766277,-73.978572,Hotel,663
2,"The Ritz-Carlton New York, Central Park",40.765272,-73.975999,Resort,319
3,The St. Regis New York,40.761339,-73.974392,Hotel,149
4,The Peninsula New York,40.761658,-73.975384,Hotel,668
5,West Side YMCA,40.770860,-73.980640,Gym / Fitness Center,109
6,Park Central Hotel New York,40.764810,-73.981400,Hotel,288
7,Gardens Suites Hotel by Affinia,40.764383,-73.963126,Hotel,406
8,Park Hyatt New York,40.765255,-73.979177,Hotel,208
9,Dream Midtown,40.764420,-73.982008,Hotel,355


Finally we can calculate the total cost of accomodation for the first hotel:

In [61]:
df_price.loc[0, 'Total cost'] = df_rides['Fare'].sum() + df_price.loc[0,'Price']*nights
print('The total cost of stay for {} is {}€'.format(df_price.loc[0, 'Name'], df_price.loc[0, 'Total cost']))

The total cost of stay for 1 Hotel Central Park is 1484.08€


Let's repeat the process for all other hotels:

In [123]:
# get airport info
airport = 'JFK Airport, New York'
airport_selection = get_airport_info(airport)

In [126]:
for ii in range(1, df_price.shape[0]):
    start = time.time()

    # select hotel
    print('loading hotel...')
    hotel_selection = df_price.loc[df_price.index==0, ['Name', 'Latitude', 'Longitude', 'Category']].copy()
    hotel_selection['Cluster'], hotel_selection['Distance'] = -1, 0.0
    print('loaded {}'.format(df_price.loc[ii, 'Name']))

    # get plan for the vacation
    print('drawing vacation plan')
    visits = tour_planner(airport_selection, hotel_selection, usr_venues, venues_plan)
    

    # get rides in transactional form
    df_rides = get_trips(visits)
    
    # calculate cost of rides
    print('calculating cab fares for {} trips...'.format(df_rides.shape[0]))
    df_rides = cab_rides_calc(df_rides)
    
    print('calculating total cost of stay')
    df_price.loc[ii, 'Total cost'] = df_rides['Fare'].sum() + df_price.loc[ii,'Price']*nights
    
    print('fine elaborazione per hotel {} - durata: {}'.format(df_price.loc[ii, 'Name'], time.time()-start))
        
        

loading hotel...
loaded Park Central Hotel New York
drawing vacation plan
calculating cab fares for 14 trips...
calculating total cost of stay
fine elaborazione per hotel Park Central Hotel New York - durata: 92.21805191040039
loading hotel...
loaded Gardens Suites Hotel by Affinia
drawing vacation plan
calculating cab fares for 14 trips...
calculating total cost of stay
fine elaborazione per hotel Gardens Suites Hotel by Affinia - durata: 90.82432961463928
loading hotel...
loaded Park Hyatt New York
drawing vacation plan
calculating cab fares for 14 trips...
calculating total cost of stay
fine elaborazione per hotel Park Hyatt New York - durata: 88.31580948829651
loading hotel...
loaded Dream Midtown
drawing vacation plan
calculating cab fares for 14 trips...
calculating total cost of stay
fine elaborazione per hotel Dream Midtown - durata: 87.87626099586487
loading hotel...
loaded The Manhattan Club
drawing vacation plan
calculating cab fares for 14 trips...
calculating total cost of

The final version of the hotel prices dataframe looks like the following:

In [68]:
df_price

,Name,Latitude,Longitude,Category,Price,Total cost
0,1 Hotel Central Park,40.765116,-73.976485,Hotel,642,1627.05
1,JW Marriott Essex House New York,40.766277,-73.978572,Hotel,663,1669.05
2,"The Ritz-Carlton New York, Central Park",40.765272,-73.975999,Resort,319,981.05
3,The St. Regis New York,40.761339,-73.974392,Hotel,149,641.05
4,The Peninsula New York,40.761658,-73.975384,Hotel,668,1679.05
5,West Side YMCA,40.770860,-73.980640,Gym / Fitness Center,109,561.05
6,Park Central Hotel New York,40.764810,-73.981400,Hotel,288,919.05
7,Gardens Suites Hotel by Affinia,40.764383,-73.963126,Hotel,406,1155.05
8,Park Hyatt New York,40.765255,-73.979177,Hotel,208,759.05
9,Dream Midtown,40.764420,-73.982008,Hotel,355,1053.05


Once again, let's save the results for later...

In [63]:
df_price.to_csv('./Resources/df_price_tot.csv', index=False)

## 8. Evaluation of results <a name="results"></a> 

In order to give recommendations about hotels, the *carpetrip* bot would use information about the budget (€ per night) of our user to make some considerations.  

Let's suppose the budget to be 200 € for example: what the solution would do is calculate the average cost, in New York, for those hotels whose price is in the neighborhood of that number.  

Finally, *carpetrip* would make his recommendation by suggesting to the user all hotels with a lower total price, highlighting in particular those accomodations whose room price is ≥ 200 €, but have a lower total cost: this because a regular tourist may have just chosen the cheaper one on paper and would have not considered this options instead.

Let's put what just said to test:

In [64]:
df_price_tot = pd.read_csv('./Resources/df_price_tot.csv')

In [69]:
df_price_tot

,Name,Latitude,Longitude,Category,Price,Total cost
0,1 Hotel Central Park,40.765116,-73.976485,Hotel,642,1627.05
1,JW Marriott Essex House New York,40.766277,-73.978572,Hotel,663,1669.05
2,"The Ritz-Carlton New York, Central Park",40.765272,-73.975999,Resort,319,981.05
3,The St. Regis New York,40.761339,-73.974392,Hotel,149,641.05
4,The Peninsula New York,40.761658,-73.975384,Hotel,668,1679.05
5,West Side YMCA,40.770860,-73.980640,Gym / Fitness Center,109,561.05
6,Park Central Hotel New York,40.764810,-73.981400,Hotel,288,919.05
7,Gardens Suites Hotel by Affinia,40.764383,-73.963126,Hotel,406,1155.05
8,Park Hyatt New York,40.765255,-73.979177,Hotel,208,759.05
9,Dream Midtown,40.764420,-73.982008,Hotel,355,1053.05


Let's make a comparison with a hotel from outside the optimal area under the assumption that our user's budget is of 200 € per night. For example let's consider the following facility:


![Solita soho](images/Comp.JPG)

Let's insert the information and repeat the previous calculations to obtain the total cost of the considered hotel.

In [73]:
hotel_comp = {'Name': 'Solita Soho Hotel', 'Latitude': 40.719902, 'Longitude': -73.999180, 'Category': 'Hotel'}
cost_per_night = 200

hotel_selection_comp = pd.DataFrame([hotel_comp], columns=['Name', 'Latitude', 'Longitude', 'Category'])
hotel_selection_comp['Cluster'], hotel_selection_comp['Distance'] = -1, 0.0

hotel_selection_comp

,Name,Latitude,Longitude,Category,Cluster,Distance
0,Solita Soho Hotel,40.719902,-73.99918,Hotel,-1,0.0


Calculating vacation plan...

In [75]:
visits_comp = tour_planner(airport_selection, hotel_selection_comp, usr_venues, venues_plan)

Obtaining fares...

In [79]:
df_rides_comp = get_trips(visits_comp)
df_rides_comp = cab_rides_calc(df_rides_comp)

Evaluating the total cost...

In [80]:
tot_cost_comp = df_rides_comp['Fare'].sum() + cost_per_night*nights

Finally, we obtain:

In [81]:
print('Total_cost = sum(fares) + cost_per_night x number_of_nights')
print('Total_cost = {} €  + {} x {} € = {} € '.format(df_rides_comp['Fare'].sum(), cost_per_night, nights, tot_cost_comp))

Total_cost = sum(fares) + cost_per_night x number_of_nights
Total_cost = 367.48 €  + 200 x 2 € = 767.48 € 


Looking at the hotels we proposed to our user we then notice that there are 3 hotels with lower total cost: 

In [87]:
df_price_tot[(df_price_tot['Total cost']<=tot_cost_comp)]

,Name,Latitude,Longitude,Category,Price,Total cost
3,The St. Regis New York,40.761339,-73.974392,Hotel,149,641.05
5,West Side YMCA,40.770860,-73.980640,Gym / Fitness Center,109,561.05
8,Park Hyatt New York,40.765255,-73.979177,Hotel,208,759.05


And in particular there is one that has a larger room price:

In [88]:
df_price_tot[(df_price_tot['Price']>cost_per_night) & (df_price_tot['Total cost']<=tot_cost_comp)]

,Name,Latitude,Longitude,Category,Price,Total cost
8,Park Hyatt New York,40.765255,-73.979177,Hotel,208,759.05


Finally we come to the conclusion that, by looking at just the price of a room (200 € vs 208 €) our user may have chosen the *Solita Soho Hotel* (4/5 rating in Trip Advior) as accomodation for his staying, but he is now able to make an upgrade to a better hotel such as the *Park Hyatt New York* (4.5/5 rating on Trip Advisor) knowing that he will spend less than in the first case since the latter is better positioned to see the sight he is interested in.

Of course, since the scope of this PoC was limited also the savings in terms of money might seem limited, but considering the overall solution it should also be noted that the user would also benefit from it in terms of time savings.  
Furthermore, scaling this solution on a larger city such as Rome or London (or even just the whole NYC) would probably bring exponentially larger benefits.